# First learning attempts
### using Beethoven dataset
* 29 pieces + transpositions across 2 octaves
* ~70h of music (2.7h per transposition)
* 0.025s resolution (40fps)

In [ ]:
IGNORE_NOTE_VELOCITY = True

# loading data files names
import os

path = '.\\datasets\\beethoven\\'
file_names = os.listdir(path)
file_names = list(filter(lambda fn: '.npz' in fn or '.npy' in fn or '.csv' in fn, file_names))
assert len(file_names) > 0, 'Data not found'

f'Found {len(file_names)} files'

In [ ]:
# loading data files
from midi_numpy.common import read_numpy_midi
file_paths = [f'{path}{fn}' for fn in file_names]

from random import choice
def load_tracks(n):
    print('loading tracks')
    sampled_file_paths = [choice(file_paths) for _ in range(n)]
    tracks = [read_numpy_midi(fp) for fp in sampled_file_paths]
    if IGNORE_NOTE_VELOCITY:
        tracks = [t[:, :128] for t in tracks]
    return tracks

In [ ]:
# select batch
def create_batch(data, batch_size, seq_length):
    # each sequence is from diffrent track
    tracks_indices = np.random.randint(0, len(data), batch_size)
    seq_indicies = [np.random.randint(0, len(data[ti]) - seq_length - 1) for ti in tracks_indices]
    # select sequences from selected tracks
    # data form is many-to-one
    x = np.stack([data[ti][si:si + seq_length] for ti,si in zip(tracks_indices, seq_indicies)])
    y = np.stack([data[ti][si + seq_length] for ti,si in zip(tracks_indices, seq_indicies)])
    return x, y

In [ ]:
# dataset generator
import numpy as np
def data_gen(batch_size, seq_len=(1, 400), track_count=25):
    # x data shape should be [batch_size, sequence_len, input_dim]
    # y shape is [batch_size, input_dim]  
    while True:
        print('reloading data')
        data = load_tracks(track_count)  
        print(f'reloaded data')
        for _ in range(1000 * len(data)):
            seq = np.random.randint(seq_len[0], seq_len[1])
            yield create_batch(data, batch_size, seq)  

## Setting up model

In [ ]:
from tensorflow import keras

INPUT_SIZE = 128 if IGNORE_NOTE_VELOCITY else 256
HIDDEN_SIZE = 512
OUTPUT_SIZE = INPUT_SIZE

BATCH_SIZE = 16

INPUT_SHAPE = (None, INPUT_SIZE)
# could be INPUT_SHAPE = (SEQUENCE_LENGTH, INPUT_SIZE)
# however predicting would have to have same seq length

In [ ]:
model = keras.models.Sequential([
    keras.layers.LSTM(HIDDEN_SIZE, input_shape=INPUT_SHAPE),
    keras.layers.Dense(OUTPUT_SIZE, activation='sigmoid')
])

model.compile(
    loss='mean_squared_error', 
    optimizer='adam', 
    metrics=['binary_accuracy']
)

In [ ]:
# or load saved model
base_path = ''
file_name = 'beth_notransp_randchunk_bcr_512_22epochs_90.0m.h5'
model = keras.models.load_model(base_path + file_name)

In [ ]:
# pre running operations
# some stat data accumultors for re-running model
from time import time
epochs_elapsed = 0
minutes_elapsed = 0
gen = data_gen(BATCH_SIZE)
test_gen = data_gen(BATCH_SIZE, track_count=3)

### Running model

In [ ]:
EPOCHS = 1
STEPS_PER_EPOCH = 1000
start_time = time()

model.fit_generator(
    gen, 
    steps_per_epoch=STEPS_PER_EPOCH, 
    epochs=EPOCHS, 
    validation_data=test_gen, 
    validation_steps=100
)

minutes_elapsed += (time() - start_time) // 60
epochs_elapsed += EPOCHS

### Saving model

In [ ]:
base_path = ''
keywords = '_'.join(['beth', 'notransp', 'randchunk'])
file_name = f'{keywords}_{HIDDEN_SIZE}_{epochs_elapsed}epochs_{minutes_elapsed}m.h5'

keras.models.save_model(model, base_path + file_name)